# Entity
Entity memory remembers given facts about specific entities in a conversation. It extracts information on entities (using an LLM) and builds up its knowledge about that entity over time (also using an LLM).

Let's first walk through using this functionality.

In [24]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationEntityMemory
llm = ChatOpenAI(model="gpt-3.5-turbo" ,temperature=0)

In [25]:
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

In [7]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Deven': 'Deven is working on a hackathon project with Sam.',
  'Sam': 'Sam is working on a hackathon project with Deven.'}}

## Using in a chain
Let's now use it in a chain!

In [26]:
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [27]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [11]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's great to hear! Hackathons can be a fantastic way to collaborate, innovate, and work on exciting projects. If you need any assistance or guidance with your hackathon project, feel free to ask!"

In [12]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam.',
 'Sam': 'Sam is working on a hackathon project with Deven.'}

In [13]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Adding more complex memory structures to Langchain sounds like a challenging but exciting endeavor! Memory structures play a crucial role in the performance and functionality of a blockchain system like Langchain. If Deven & Sam need any assistance or insights on how to implement these complex memory structures, feel free to ask for help. I'm here to provide guidance and support throughout the process."

In [14]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's a great addition! Implementing a key-value store for entities mentioned in the conversation can help with efficient data storage and retrieval. It can also enhance the overall functionality and performance of the system. If Deven & Sam need any assistance or guidance on integrating this key-value store into Langchain, feel free to ask for help. I'm here to provide support and insights on how to effectively incorporate this feature into their project."

In [15]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'Deven and Sam are working on a hackathon project together, specifically focusing on adding more complex memory structures to Langchain. They are also integrating a key-value store for entities mentioned in their conversation, which can enhance data storage and retrieval efficiency. If you have any specific questions or need more information about Deven & Sam, feel free to ask!'

## Inspecting the memory store
We can also inspect the memory store directly. In the following examples, we look at it directly, and then go through some examples of adding information and watch how it changes.

In [17]:
print(conversation.memory.entity_store.store)

{'Deven': 'Deven is working on a hackathon project with Sam and they are adding a key-value store for entities mentioned in their conversation.', 'Sam': 'Sam is working on a hackathon project with Deven and is integrating a key-value store for entities mentioned in their conversation.', 'Langchain': 'Langchain is adding in a key-value store for entities mentioned so far in the conversation.'}


In [16]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Deven': 'Deven is working on a hackathon project with Sam and they are '
          'adding a key-value store for entities mentioned in their '
          'conversation.',
 'Langchain': 'Langchain is adding in a key-value store for entities mentioned '
              'so far in the conversation.',
 'Sam': 'Sam is working on a hackathon project with Deven and is integrating a '
        'key-value store for entities mentioned in their conversation.'}


In [18]:
conversation.predict(input="Sam is the founder of a company called Daimon.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's interesting to know! Sam being the founder of a company called Daimon adds another layer to their background and expertise. It's great to see individuals like Sam involved in innovative projects like the one they are working on with Deven. If you have any more information or questions about Sam or Daimon, feel free to share!"

In [21]:
print(conversation.memory.entity_store.store)

{'Deven': 'Deven is working on a hackathon project with Sam and they are adding a key-value store for entities mentioned in their conversation.', 'Sam': 'Sam is working on a hackathon project with Deven and is integrating a key-value store for entities mentioned in their conversation.', 'Langchain': 'Langchain is adding in a key-value store for entities mentioned so far in the conversation.', 'Daimon': 'Daimon is a company founded by Sam.'}


In [22]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Daimon': 'Daimon is a company founded by Sam.',
 'Deven': 'Deven is working on a hackathon project with Sam and they are '
          'adding a key-value store for entities mentioned in their '
          'conversation.',
 'Langchain': 'Langchain is adding in a key-value store for entities mentioned '
              'so far in the conversation.',
 'Sam': 'Sam is working on a hackathon project with Deven and is integrating a '
        'key-value store for entities mentioned in their conversation.'}


In [23]:
conversation.predict(input="What do you know about Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Sam is working on a hackathon project with Deven and is the founder of a company called Daimon. Sam's involvement in innovative projects and founding a company like Daimon showcases their background and expertise in the field. If you have any specific questions or need more information about Sam, feel free to ask!"